## Supervised Learning: Classifying Sunny and Rainy Day Songs

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import requests
import pandas as pd
import numpy as np
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks

from colorthief import ColorThief
from urllib.request import urlopen
import io

from mvspotifyhelper.mvspotifyhelper import MV

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay

In [2]:
dotenv.load_dotenv()

username = 'michael_vaden'

spot_id = os.getenv('spot_id')
spot_secret = os.getenv('spot_secret')
redirect_uri= 'https://www.virginia.edu/'

client_credentials_manager = SpotifyClientCredentials(client_id=spot_id, client_secret=spot_secret)

scope = "playlist-modify-public playlist-modify-private playlist-read-private playlist-read-collaborative user-library-modify"

token = util.prompt_for_user_token(username, scope, spot_id, spot_secret, redirect_uri, show_dialog=True)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=token)

### Select Sunny and Rainy Themed Playlists

*To begin and make sure that this approach holds water*, I selected 3-4 playlists that were publicly available on Spotify for each category of sunny and rainy. These playlists all had more than 10k followers, and were named appropriately (rainy day, rainy days music, sunny day, summer party, etc). Many of these playlists are authored by Spotify. When I had roughly 700 total songs and an approximately balanced dataset, I used my helper functions from my built-in module (mvspotifyhelper) to extract the tracks (songs) and track features from these playlists to compile into a **training set**.

In [3]:
rainy1 = MV.get_tracks_from_url('spotify', "https://open.spotify.com/playlist/37i9dQZF1DXbvABJXBIyiY?si=e2da83ce834b40c5") # rainy day
rainy2 = MV.get_tracks_from_url('Circles Records', "https://open.spotify.com/playlist/3r82Jvzw3SSGKKiKf3dXMM?si=01242f93e6524a23").loc[:210,] # rainy days music
rainy3 = MV.get_tracks_from_url('tiarafernando', "https://open.spotify.com/playlist/47S4MBG0EEXwA0GdJUA4Ur?si=0e19386208f944c3") # a playlist for rainy days

In [4]:
sunny1 = MV.get_tracks_from_url('spotify', "https://open.spotify.com/playlist/37i9dQZF1DX1BzILRveYHb?si=ac80c28e8a104400") # sunny day
sunny2 = MV.get_tracks_from_url('spotify', "https://open.spotify.com/playlist/37i9dQZF1DXd1MXcE8WTXq?si=8662accbcb644c0e") # summer throwbacks
sunny3 = MV.get_tracks_from_url('spotify', "https://open.spotify.com/playlist/37i9dQZF1DX5Ozry5U6G0d?si=953f1b53a9584ce7") # summer party
sunny4 = MV.get_tracks_from_url('spotify', 'https://open.spotify.com/playlist/37i9dQZF1DX9fZ7amiNVu6?si=bf527d1910ff4453') # feel good summer

In [5]:
rainy = pd.concat([rainy1, rainy2, rainy3]).drop_duplicates('track.uri')
#rainy

In [6]:
sunny = pd.concat([sunny1, sunny2, sunny3, sunny4]).drop_duplicates('track.uri')
#sunny

In [7]:
#rainy.describe()
rainy = rainy[['track.uri', 'track.id', 'track.name', 'track.popularity']].dropna()

In [8]:
#sunny.describe()
sunny = sunny[['track.uri', 'track.id', 'track.name', 'track.popularity']].dropna()

In [9]:
sunny_features = MV.add_song_features(sunny).drop(['id', 'uri', 'track_href', 'analysis_url', 'type'], axis=1)
rainy_features = MV.add_song_features(rainy).drop(['id', 'uri', 'track_href', 'analysis_url', 'type'], axis=1)

In [10]:
sunny_features['weather_type'] = 'sun'
rainy_features['weather_type'] = 'rain'
weather_songs = pd.concat([sunny_features, rainy_features])
weather_songs

,track.uri,track.id,track.name,track.popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,weather_type
0,spotify:track:3ZpQiJ78LKINrW9SQTgbXd,3ZpQiJ78LKINrW9SQTgbXd,All I Wanna Do,74.0,0.820,0.528,9,-11.179,1,0.0321,0.1110,0.018600,0.2570,0.9310,120.091,272107,4,sun
1,spotify:track:3XKIUb7HzIF1Vu9usunMzc,3XKIUb7HzIF1Vu9usunMzc,Maria Maria (feat. The Product G&B),83.0,0.777,0.601,2,-5.931,1,0.1260,0.0406,0.002010,0.0348,0.6800,97.911,261973,4,sun
2,spotify:track:0ofHAoxe9vBkTCp2UQIavz,0ofHAoxe9vBkTCp2UQIavz,Dreams - 2004 Remaster,87.0,0.828,0.492,0,-9.744,1,0.0276,0.0644,0.004280,0.1280,0.7890,120.151,257800,4,sun
3,spotify:track:0bRXwKfigvpKZUurwqAlEh,0bRXwKfigvpKZUurwqAlEh,Lovely Day,81.0,0.692,0.651,9,-8.267,1,0.0324,0.2920,0.002410,0.1050,0.7060,97.923,254560,4,sun
4,spotify:track:1YLJVmuzeM2YSUkCCaTNUB,1YLJVmuzeM2YSUkCCaTNUB,Dog Days Are Over,79.0,0.492,0.810,7,-5.315,1,0.0847,0.0416,0.003790,0.1170,0.2450,149.954,251840,4,sun
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,spotify:track:39sDitIeCMrVX2QyXHY46t,39sDitIeCMrVX2QyXHY46t,Blue Hair,85.0,0.751,0.720,4,-6.376,1,0.0303,0.5540,0.049700,0.2580,0.8840,135.730,217000,4,rain
387,spotify:track:3Q4zjvPKaxkGUo0MTdrbir,3Q4zjvPKaxkGUo0MTdrbir,Stereotypes,34.0,0.677,0.475,11,-6.113,1,0.0308,0.5320,0.000000,0.2030,0.6120,117.822,168424,4,rain
388,spotify:track:1MHb9ROmrgyGvPXx68i04u,1MHb9ROmrgyGvPXx68i04u,Vulnerable,44.0,0.598,0.183,9,-13.487,1,0.0304,0.5350,0.000062,0.1290,0.0796,137.035,155146,4,rain
389,spotify:track:1jCcfSHdXosT54e64GFRE1,1jCcfSHdXosT54e64GFRE1,Deceive Me So Easy,3.0,0.400,0.457,9,-10.543,0,0.0404,0.3680,0.006790,0.3150,0.1390,77.182,238213,4,rain


In [11]:
weather_for_ML = weather_songs.iloc[:,3:]
weather_for_ML['weather_type'] = weather_for_ML['weather_type'].map({'rain':0, 'sun':1})
weather_for_ML

,track.popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,weather_type
0,74.0,0.820,0.528,9,-11.179,1,0.0321,0.1110,0.018600,0.2570,0.9310,120.091,272107,4,1
1,83.0,0.777,0.601,2,-5.931,1,0.1260,0.0406,0.002010,0.0348,0.6800,97.911,261973,4,1
2,87.0,0.828,0.492,0,-9.744,1,0.0276,0.0644,0.004280,0.1280,0.7890,120.151,257800,4,1
3,81.0,0.692,0.651,9,-8.267,1,0.0324,0.2920,0.002410,0.1050,0.7060,97.923,254560,4,1
4,79.0,0.492,0.810,7,-5.315,1,0.0847,0.0416,0.003790,0.1170,0.2450,149.954,251840,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,85.0,0.751,0.720,4,-6.376,1,0.0303,0.5540,0.049700,0.2580,0.8840,135.730,217000,4,0
387,34.0,0.677,0.475,11,-6.113,1,0.0308,0.5320,0.000000,0.2030,0.6120,117.822,168424,4,0
388,44.0,0.598,0.183,9,-13.487,1,0.0304,0.5350,0.000062,0.1290,0.0796,137.035,155146,4,0
389,3.0,0.400,0.457,9,-10.543,0,0.0404,0.3680,0.006790,0.3150,0.1390,77.182,238213,4,0


We need to account for the discrete variables that we have in our dataset. Specifically, we need to make sure that key, mode (major or minor), and time signature are all treated as categorical descriptions of songs rather than numerical ones.

In [12]:
weather_for_ML[['key', 'mode', 'time_signature', 'weather_type']] = weather_for_ML[['key', 'mode', 'time_signature', 'weather_type']].astype('category')

weather_for_ML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 701 entries, 0 to 390
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   track.popularity  701 non-null    float64 
 1   danceability      701 non-null    float64 
 2   energy            701 non-null    float64 
 3   key               701 non-null    category
 4   loudness          701 non-null    float64 
 5   mode              701 non-null    category
 6   speechiness       701 non-null    float64 
 7   acousticness      701 non-null    float64 
 8   instrumentalness  701 non-null    float64 
 9   liveness          701 non-null    float64 
 10  valence           701 non-null    float64 
 11  tempo             701 non-null    float64 
 12  duration_ms       701 non-null    int64   
 13  time_signature    701 non-null    category
 14  weather_type      701 non-null    category
dtypes: category(4), float64(10), int64(1)
memory usage: 69.3 KB


### Supervised Learning Classification Models

After compiling a complete dataset of roughly 700 songs that can be categorized as 'sun' or 'rain' songs, I created a train and test set from this data and used multiple machine learning approaches to attempt to correctly classify these songs.

In [13]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(weather_for_ML, test_size=0.2, random_state=42)

#test_set, validation_set = train_test_split(test_set, test_size = 0.5, random_state=42)

X_train = train_set.drop("weather_type", axis=1)
y_train = train_set["weather_type"].copy()

X_test = test_set.drop("weather_type", axis=1)
y_test = test_set["weather_type"].copy()

#X_validation = validation_set.drop("weather_type", axis=1)
#y_validation = validation_set["weather_type"].copy()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(560, 14)
(141, 14)
(560,)
(141,)


To improve the performance of some of our classifier models, we standardize all of our numerical values in the dataset and one-hot-encode each of our categorical variables.

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

num_cols = list(X_train.select_dtypes(include=[np.number]))
cat_cols = list(X_train.select_dtypes(exclude=[np.number])) # key, mode, time signature

pipeline=ColumnTransformer([
    ('num',StandardScaler(),num_cols),
    ('cat',OneHotEncoder(handle_unknown="ignore"),cat_cols),
])

X_train = pipeline.fit_transform(X_train)

X_test = pipeline.transform(X_test)

Below are *Random Forest, XGBoost, AdaBoost, and Neural Network* approaches. Decision Trees, Logistic Regression, and Naive Bayes were also implemented, but the methods included yielded the best initial results. In the context of this classification of sunny and rainy songs, there is no significant consequence to false positive or false negative errors comparatively. As a result, for the first iteration of models, our measure of choice is simply accuracy. 

### Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import randint 

forest_cls = RandomForestClassifier()

param_dist = {
  'n_estimators': randint(low=50, high=200),
  'max_features': randint(low=5, high=15),
  'max_depth': randint(low=5, high=15)
}

forest_rnd_search = RandomizedSearchCV(
    estimator=forest_cls, 
    param_distributions=param_dist,
    n_iter=50, 
    cv=5
)

forest_rnd_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=50,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fb3aac1d160>,
                                        'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fb3aac18280>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fb3aa9ef4f0>})

In [20]:
print(forest_rnd_search.best_params_)

{'max_depth': 8, 'max_features': 10, 'n_estimators': 185}


In [21]:
y_predict_RF = forest_rnd_search.best_estimator_.predict(X_test)

In [22]:

accuracy_score(y_predict_RF, y_test)

0.8865248226950354

In [23]:
confusion_matrix(y_predict_RF, y_test)

array([[68,  9],
       [ 7, 57]])

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=10, shuffle=True)
forest_scores = cross_val_score(forest_rnd_search.best_estimator_, weather_for_ML.drop('weather_type', axis=1).copy(), 
                              weather_for_ML['weather_type'], scoring='accuracy', cv=cv)

print("Mean:", forest_scores.mean())
print("Standard deviation:", forest_scores.std())

Mean: 0.8701207243460765
Standard deviation: 0.02763847176662921


### XGBoost

In [25]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


param_grid = {
    "max_depth": [i for i in range(5,16)],
    "learning_rate": [.001, .005, .01, .05, .1, .2, .3, .5, 1],
    "gamma": [0, 0.25, 0.5, 1],
    "reg_lambda": [0, 1],
    "scale_pos_weight": [i for i in range(1,3)],
    "subsample": [i/10.0 for i in range(6,10)],
    "colsample_bytree": [i/10.0 for i in range(6,10)],
}

xgb_cls = xgb.XGBClassifier(tree_method="hist", enable_categorical=True)

xgb_rnd_search = RandomizedSearchCV(
    estimator=xgb_cls, 
    param_distributions=param_grid, 
    n_iter=200,
    cv=5
)

xgb_rnd_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=True,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=...
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=200,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8,
                                                             0.9],
                                        'gamma': [0, 0.25, 0.5, 1],
                                        'learning_rate': [0.001, 0.005, 0.01,
                                                          0.05, 0.1, 0.2, 0.3,
                                                          0.5, 1],
                                        'max_depth': [5, 6, 7, 8, 9, 10, 11, 12,
                                                      13, 14, 15],
                                        'reg_lambda': [0, 1],
                                        'scale_pos_weight': [1, 2],
                                        'subsample': [0.6, 0.7, 0.8, 0.9]})

In [26]:
print(xgb_rnd_search.best_params_)

y_predict_XG = xgb_rnd_search.best_estimator_.predict(X_test)

accuracy_score(y_predict_XG, y_test)

{'subsample': 0.7, 'scale_pos_weight': 1, 'reg_lambda': 1, 'max_depth': 13, 'learning_rate': 0.5, 'gamma': 1, 'colsample_bytree': 0.8}


0.8723404255319149

In [27]:
cv = KFold(n_splits=10, shuffle=True)
xgb_scores = cross_val_score(xgb_rnd_search.best_estimator_, weather_for_ML.drop('weather_type', axis=1).copy(), 
                              weather_for_ML['weather_type'], scoring='accuracy', cv=cv)

print("Mean:", xgb_scores.mean())
print("Standard deviation:", xgb_scores.std())

Mean: 0.8716096579476862
Standard deviation: 0.0383364433150584


### Adaboost

In [28]:
from sklearn.ensemble import AdaBoostClassifier

ada_cls = AdaBoostClassifier()


param_grid = {
    'n_estimators': [i*10 for i in range(1, 11)],
    'learning_rate': [i/100 for i in range(1, 101)]
}

ada_rnd_search = RandomizedSearchCV(
    estimator=ada_cls, 
    param_distributions=param_grid, 
    n_iter=200,
    cv=5
)

ada_rnd_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=AdaBoostClassifier(), n_iter=200,
                   param_distributions={'learning_rate': [0.01, 0.02, 0.03,
                                                          0.04, 0.05, 0.06,
                                                          0.07, 0.08, 0.09, 0.1,
                                                          0.11, 0.12, 0.13,
                                                          0.14, 0.15, 0.16,
                                                          0.17, 0.18, 0.19, 0.2,
                                                          0.21, 0.22, 0.23,
                                                          0.24, 0.25, 0.26,
                                                          0.27, 0.28, 0.29, 0.3, ...],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90, 100]})

In [29]:
print(ada_rnd_search.best_params_)

y_predict_AD = ada_rnd_search.best_estimator_.predict(X_test)

print(accuracy_score(y_predict_AD, y_test))

{'n_estimators': 100, 'learning_rate': 0.32}
0.8865248226950354


In [30]:
cv = KFold(n_splits=10, shuffle=True)
ada_scores = cross_val_score(ada_rnd_search.best_estimator_, weather_for_ML.drop('weather_type', axis=1).copy(), 
                              weather_for_ML['weather_type'], scoring='accuracy', cv=cv)

print("Mean:", ada_scores.mean())
print("Standard deviation:", ada_scores.std())

Mean: 0.8801609657947684
Standard deviation: 0.029451297448768173


### Neural Network

In [31]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow import keras

tf.keras.utils.set_random_seed(42)

def run_neural_net(X_train, y_train, X_test, y_test):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(29,)),
        keras.layers.Dense(20, activation=tf.nn.relu),
        keras.layers.Dense(12, activation=tf.nn.relu),
        keras.layers.Dense(1, activation=tf.nn.sigmoid),
    ])

    model.compile( loss= "binary_crossentropy",

                    optimizer = tf.keras.optimizers.Adam(),

                    metrics = ['accuracy'])

    model.fit(X_train, y_train, epochs=10, batch_size=5, verbose=2)

    loss, accuracy = model.evaluate(X_test, y_test)

    return [loss, accuracy, model.predict(X_test)]

nn1 = run_neural_net(X_train, y_train, X_test, y_test)

print(f' Model loss on the test set: {nn1[0]}')
print(f' Model accuracy on the test set: {100*nn1[1]}')

2023-08-18 21:03:02.834758: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
112/112 - 1s - loss: 0.6129 - accuracy: 0.6286 - 1s/epoch - 11ms/step
Epoch 2/10
112/112 - 0s - loss: 0.4521 - accuracy: 0.8196 - 152ms/epoch - 1ms/step
Epoch 3/10
112/112 - 0s - loss: 0.3573 - accuracy: 0.8607 - 157ms/epoch - 1ms/step
Epoch 4/10
112/112 - 0s - loss: 0.3133 - accuracy: 0.8696 - 151ms/epoch - 1ms/step
Epoch 5/10
112/112 - 0s - loss: 0.2939 - accuracy: 0.8875 - 158ms/epoch - 1ms/step
Epoch 6/10
112/112 - 0s - loss: 0.2822 - accuracy: 0.8821 - 170ms/epoch - 2ms/step
Epoch 7/10
112/112 - 0s - loss: 0.2746 - accuracy: 0.8911 - 185ms/epoch - 2ms/step
Epoch 8/10
112/112 - 0s - loss: 0.2661 - accuracy: 0.8946 - 158ms/epoch - 1ms/step
Epoch 9/10
112/112 - 0s - loss: 0.2602 - accuracy: 0.8964 - 150ms/epoch - 1ms/step
Epoch 10/10
112/112 - 0s - loss: 0.2542 - accuracy: 0.8982 - 152ms/epoch - 1ms/step
5/5 [==============================] - 0s 2ms/step
 Model loss on the test set: 0.2581961154937744
 Model accuracy on the test set: 92.90780425071716


In [32]:
confusion_matrix(np.rint(nn1[2]), y_test)

array([[68,  3],
       [ 7, 63]])

Based on the models used above, we can see that the results for the random forest, XGBoost, and AdaBoost approaches are all very similar. The neural network appears to perform the best out of all of the models, although it was not cross-validated like the other approaches. 

Other models such as logistic regression, decision trees, and naive bayes were attempted. However, the ones above yielded the best initial results. 

There are many potential next steps, including exploring more advanced hypertuning of the above models, cross-validating the neural network, and exploring new options such as KNN and advanced deep learning models. However, because accuracy here only applies to our 700 songs, I want to implement a weather playlist generator and see the song results before I iterate further. Based on the results of the playlists that are generated, I can see if the data that was used to train these models needs to be increased or changed, and determine if there is a better measure for performance than accuracy.